# Simple iteration for systems of linear equations

First, generate a random diagonally dominant matrix, for testing.

In [2]:
import numpy as np
rndm = np.random.RandomState(1234)

n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)

# I.  Jacobi iteration

Given

$$
A x = b
$$

separate the diagonal part $D$,

$$ A = D + (A - D) $$

and write

$$
x = D^{-1} (D - A) x + D^{-1} b\;.
$$

Then iterate

$$
x_{n + 1} = B x_{n} + c\;,
$$

where 

$$
B = D^{-1} (A - D) \qquad \text{and} \qquad c = D^{-1} b
$$


Let's construct the matrix and the r.h.s. for the Jacobi iteration

In [3]:
diag_1d = np.diag(A)

B = -A.copy()
np.fill_diagonal(B, 0)

D = np.diag(diag_1d)
invD = np.diag(1./diag_1d)
BB = invD @ B 
c = invD @ b

In [4]:
# sanity checks
from numpy.testing import assert_allclose

assert_allclose(-B + D, A)


# xx is a "ground truth" solution, compute it using a direct method
xx = np.linalg.solve(A, b)

np.testing.assert_allclose(A@xx, b)
np.testing.assert_allclose(D@xx, B@xx + b)
np.testing.assert_allclose(xx, BB@xx + c)

Check that $\| B\| \leqslant 1$:

In [5]:
np.linalg.norm(BB)

0.36436161983015336

### Do the Jacobi iteration

In [6]:
n_iter = 50

x0 = np.ones(n)
x = x0
for _ in range(n_iter):
    x = BB @ x + c

In [7]:
# Check the result:

A @ x - b

array([ 1.11022302e-16,  0.00000000e+00, -2.22044605e-16, -1.11022302e-16,
        1.11022302e-16,  0.00000000e+00, -2.42861287e-17,  0.00000000e+00,
       -2.77555756e-17,  1.11022302e-16])

### Task I.1

Collect the proof-of-concept above into a single function implementing the Jacobi iteration. This function should receive the r.h.s. matrix $A$, the l.h.s. vector `b`, and the number of iterations to perform.


The matrix $A$ in the illustration above is strongly diagonally dominant, by construction. 
What happens if the diagonal matrix elements of $A$ are made smaller? Check the convergence of the Jacobi iteration, and check the value of the norm of $B$.

(20% of the total grade)


In [8]:
# ... ENTER YOUR CODE HERE ...

def J_iteration(A, b, eps=1e-12, iter=50): 
    Diag_1 = np.diag(A)  
    B = -A.copy()

    np.fill_diagonal(B, 0) 
    invD = np.diag(1.0 / Diag_1)
    
    BB = invD @ B 
    c = invD @ b 
    
    x = np.ones(n)
    for _ in range(n_iter): x = BB @ x + c

    return x 

In [9]:
x = J_iteration(A, b)
print(x)
np.testing.assert_allclose(A@x, b)
np.testing.assert_allclose(x, xx)

[ 0.03919429  0.03780037  0.04283232  0.02365951  0.05745031 -0.00030244
 -0.00577279  0.03177549 -0.00422849  0.05284648]


In [10]:
for k in range(1, 12):
    A1 = A + np.diagflat([-k]*n)
    print(np.linalg.norm(np.diag(1./np.diag(A1))@(-A1.copy()+np.diag(np.diag(A1)))),
          np.linalg.norm(J_iteration(A1, b)-np.linalg.solve(A1, b)))

0.38959181027260875 2.009053247780777e-17
0.4185783948614869 2.7449806536241494e-17
0.4522284025473819 4.7730515306684666e-17
0.4917667095178099 2.3354429532350847e-17
0.5388887887486234 1.8193933307329566e-17
0.5960110344093966 1.036960250308455e-15
0.6667001660296402 2.7708368486989536e-13
0.7564517359241753 1.4990528448342257e-10
0.8742017351588476 2.016159609243833e-07
1.0355299928250665 0.0009191717405677776
1.2702850939751231 23.48163367953915


# II. Seidel's iteration.

##### Task II.1

Implement the Seidel's iteration. 

Test it on a random matrix. Study the convergence of iterations, relate to the norm of the iteration matrix.

(30% of the total grade)

In [11]:
# ... ENTER YOUR CODE HERE ...

def Seidels_iteration(A, b, eps=1e-12, iter=100):

    x = np.zeros_like(b, dtype=np.double)
    
    for k in range(iter):
        
        x_old  = x.copy()
        
        for i in range(A.shape[0]):
            x[i] = (b[i] - np.dot(A[i,:i], x[:i]) - np.dot(A[i,(i+1):], x_old[(i+1):])) / A[i ,i]
      
    return x  
        

In [12]:
x = Seidels_iteration(A, b)

In [13]:
np.testing.assert_allclose(x, xx)
np.testing.assert_allclose(A @ x, b)

In [22]:
for k in range(1, 16):
    A1 = A + np.diagflat([-k]*n)
    print(np.linalg.norm(np.diag(1./np.diag(A1))@(-A1.copy()+np.diag(np.diag(A1)))),
          np.linalg.norm(Seidels_iteration(A1, b)-np.linalg.solve(A1, b)))

0.39315162237575607 1.804862860440849e-17
0.42202631543251723 2.3864272523720094e-17
0.45548069473587693 1.97026712507607e-17
0.4946988077730063 2.124593680485185e-17
0.541311664415742 1.274756208291111e-17
0.5976303078076515 3.382710778154774e-17
0.6670418762840838 3.104379629778137e-17
0.7547193781175311 3.933129244866141e-17
0.8689797529637481 2.964296775167217e-17
1.0241070526789309 3.7043556597280256e-17
1.2468971279649803 5.1925927263190304e-17
1.594301462446768 5.1460214472274106e-17
2.213124726732211 1.2590794595551717e-16
3.6414476395758366 0.0002698055026768034
11.669681525331226 5.748747416973585e+70


# III. Minimum residual scheme

### Task III.1

Implement the $\textit{minimum residual}$ scheme: an explicit non-stationary method, where at each step you select the iteration parameter $\tau_n$ to minimize the residual $\mathbf{r}_{n+1}$ given $\mathbf{r}_n$. Test it on a random matrix, study the convergence to the solution, in terms of the norm of the residual and the deviation from the ground truth solution (which you can obtain using a direct method). Study how the iteration parameter $\tau_n$ changes as iterations progress.

(50% of the grade)

In [23]:
# ... ENTER YOUR CODE HERE ...

def min_iter(A, b, show_plot=True, eps=1e-12, maxiter=100):
    m, n = A.shape 
    x0 = np.ones(n)
    tau_series = []
    
    for i in range(maxiter): 
        r = np.matmul(A, x0) - b
        temp = np.matmul(A, r)
        tau = np.matmul(r, temp) / np.linalg.norm(temp, ord=2)**2
        tau_series.append(tau)
        x = x0 - tau * r
        
        if np.linalg.norm(x - x0) < eps: 
            break
        
        x0 = x
    
    return x

In [24]:
x = min_iter(A, b)

In [25]:
np.testing.assert_allclose(x, xx)
np.testing.assert_allclose(A @ x, b)

In [26]:
for k in range(1, 16):
    A1 = A + np.diagflat([-k]*n)
    print(np.linalg.norm(np.diag(1./np.diag(A1))@(-A1.copy()+np.diag(np.diag(A1)))),
          np.linalg.norm(min_iter(A1, b)-np.linalg.solve(A1, b)))

0.39315162237575607 9.626948947857578e-14
0.42202631543251723 4.8560675831026595e-14
0.45548069473587693 1.4155778324491284e-13
0.4946988077730063 8.943784595623082e-14
0.541311664415742 8.602895641992291e-14
0.5976303078076515 9.84148716555474e-14
0.6670418762840838 1.6961640035576557e-13
0.7547193781175311 1.2311990868858909e-13
0.8689797529637481 1.5872779475771453e-13
1.0241070526789309 3.749836480536345e-13
1.2468971279649803 2.8994072849521496e-13
1.594301462446768 7.221252832963951e-13
2.213124726732211 1.2822882034613518e-12
3.6414476395758366 1.354884953488899e-07
11.669681525331226 1.0684768905349105
